In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/Deep Learning Projects/Omdena/Morphological modelling

/content/gdrive/MyDrive/Deep Learning Projects/Omdena/Morphological modelling


In [ ]:
!unzip Morphological-API.zip

Archive:  Morphological-API.zip
replace __init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install huggingface-hub==0.0.12
!pip install tokenizers==0.10.3
!pip install transformers==4.8.2
!pip install farasapy==0.0.13
!pip install pyarabic==0.6.11
!pip install flask==1.1.4
!pip install gdown==3.13.0
!pip install joblib

     |████████████████████████████████| 2.5MB 5.1MB/s 
  Found existing installation: transformers 4.9.0.dev0
    Uninstalling transformers-4.9.0.dev0:
      Successfully uninstalled transformers-4.9.0.dev0


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.13.0-cp37-none-any.whl size=9046 sha256=0150b2adf7c19bc48fdf1e680bc8c761917f0623601927b07815d6087be9d21f
  Stored in directory: /root/.cache/pip/wheels/ba/fa/c5/12813d7496f34652c43a471e11a780e769889d06e34735c32e
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4


# To run it using Colab, add this code in app.py


```
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app
```


In [ ]:
!python app.py


2021-07-09 14:02:35.320463: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://934396d03f5b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [09/Jul/2021 14:02:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2021 14:02:42] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
  0% 0.00/241M [00:00<?, ?iB/s]/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS reque

Press on this link:
http://934396d03f5b.ngrok.io

